# Quantum ESPRESSO Workshop - Part 1: Introduction and Setup

## Learning Objectives
1. Understand what Quantum ESPRESSO solves (DFT, Kohn-Sham equations)
2. Verify the computational environment
3. Understand the structure of QE input files
4. Set up the working directory and pseudopotentials

---

## 1. What is Quantum ESPRESSO?

Quantum ESPRESSO (QE) is an integrated suite of open-source codes for:
- **Electronic structure calculations** using Density Functional Theory (DFT)
- **Plane-wave basis sets** with periodic boundary conditions
- **Pseudopotentials** to describe electron-ion interactions

### The Kohn-Sham Equations

DFT maps the many-electron problem onto a set of single-particle equations:

$$\left[-\frac{\hbar^2}{2m}\nabla^2 + V_{eff}(\mathbf{r})\right]\psi_i(\mathbf{r}) = \varepsilon_i\psi_i(\mathbf{r})$$

where $V_{eff}$ includes:
- External potential (from ions)
- Hartree potential (electron-electron Coulomb)
- Exchange-correlation potential (approximated)

### Key Approximations
1. **Exchange-correlation functional**: LDA, GGA (PBE), hybrid functionals
2. **Pseudopotential**: Replaces core electrons with effective potential
3. **Plane-wave basis**: Requires kinetic energy cutoff (ecutwfc)
4. **k-point sampling**: Discretizes the Brillouin zone

---

## 2. Environment Verification

Let's verify that Quantum ESPRESSO and all required tools are properly installed.

In [ ]:
import subprocess
import sys
import os
from pathlib import Path

def check_command(cmd, name):
    """Check if a command is available in the system."""
    result = subprocess.run(['which', cmd], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ {name}: {result.stdout.strip()}")
        return True
    else:
        print(f"✗ {name}: NOT FOUND")
        return False

print("=" * 50)
print("Quantum ESPRESSO Environment Check")
print("=" * 50)

# Check QE executables
qe_tools = [
    ('pw.x', 'PWscf (plane-wave self-consistent field)'),
    ('bands.x', 'Band structure post-processing'),
    ('dos.x', 'Density of states'),
    ('projwfc.x', 'Projected DOS'),
    ('pp.x', 'Post-processing'),
    ('plotband.x', 'Band plotting utility')
]

print("\nQuantum ESPRESSO executables:")
print("-" * 30)
all_found = True
for cmd, desc in qe_tools:
    if not check_command(cmd, desc):
        all_found = False

if all_found:
    print("\n✓ All QE executables found!")
else:
    print("\n⚠ Some executables missing. Install with: apt install quantum-espresso")

In [ ]:
# Check Python packages
print("\nPython packages:")
print("-" * 30)

packages = ['numpy', 'matplotlib', 'ase']
for pkg in packages:
    result = subprocess.run([sys.executable, '-c', f'import {pkg}; print({pkg}.__version__)'],
                          capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ {pkg}: {result.stdout.strip()}")
    else:
        print(f"✗ {pkg}: NOT FOUND")
        print(f"  Install with: pip install {pkg}")

In [ ]:
# Get QE version
print("\nQuantum ESPRESSO Version:")
print("-" * 30)
result = subprocess.run(['pw.x', '--version'], capture_output=True, text=True, timeout=10)
# pw.x prints version info to stdout when run with --version or even without input
for line in result.stdout.split('\n')[:5]:
    if line.strip():
        print(line)

---

## 3. Directory Structure Setup

We'll organize our calculations in a clean directory structure:

In [ ]:
# Define the workshop root directory
# For Google Colab, change this to '/content/qe_workshop'
# For local Jupyter, use a path in your home directory

WORKSHOP_ROOT = Path.cwd().parent  # Assumes notebook is in 'notebooks' subdirectory

# If running standalone, set absolute path
if not (WORKSHOP_ROOT / 'pseudopotentials').exists():
    WORKSHOP_ROOT = Path('/home/claude/qe_workshop')

# Define directory paths
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'
STRUCT_DIR = WORKSHOP_ROOT / 'structures'

# Create directories if they don't exist
for dir_path in [PSEUDO_DIR, OUTPUT_DIR, STRUCT_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"Workshop root: {WORKSHOP_ROOT}")
print(f"Pseudopotential directory: {PSEUDO_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Structures directory: {STRUCT_DIR}")

# Set environment variable for QE
os.environ['PSEUDO_DIR'] = str(PSEUDO_DIR)
print(f"\nPSEUDO_DIR environment variable set to: {os.environ['PSEUDO_DIR']}")

---

## 4. Pseudopotential Setup

Pseudopotentials are crucial for plane-wave DFT calculations. We'll use the **PBE** exchange-correlation functional with **PAW** (Projector Augmented Wave) pseudopotentials from the **PSlibrary**.

### Why PAW pseudopotentials?
- Accurate description of core-valence interactions
- All-electron precision with pseudopotential efficiency
- Well-tested and widely used in the community

In [ ]:
import urllib.request

# Pseudopotential URLs from the official QE repository
PSEUDOPOTENTIALS = {
    'Si': {
        'filename': 'Si.upf',
        'url': 'https://pseudopotentials.quantum-espresso.org/upf_files/Si.pbe-n-kjpaw_psl.1.0.0.UPF',
        'description': 'Silicon PAW-PBE pseudopotential',
        'ecutwfc_suggested': 40.0,  # Ry
        'ecutrho_suggested': 320.0  # Ry (8x ecutwfc for PAW)
    }
}

def download_pseudopotential(element):
    """Download pseudopotential if not already present."""
    info = PSEUDOPOTENTIALS[element]
    filepath = PSEUDO_DIR / info['filename']
    
    if filepath.exists():
        size_mb = filepath.stat().st_size / (1024 * 1024)
        print(f"✓ {info['filename']} already exists ({size_mb:.2f} MB)")
        return filepath
    
    print(f"Downloading {info['filename']}...")
    urllib.request.urlretrieve(info['url'], filepath)
    size_mb = filepath.stat().st_size / (1024 * 1024)
    print(f"✓ Downloaded {info['filename']} ({size_mb:.2f} MB)")
    return filepath

# Download Silicon pseudopotential
si_pp = download_pseudopotential('Si')
print(f"\nPseudopotential path: {si_pp}")
print(f"Suggested ecutwfc: {PSEUDOPOTENTIALS['Si']['ecutwfc_suggested']} Ry")
print(f"Suggested ecutrho: {PSEUDOPOTENTIALS['Si']['ecutrho_suggested']} Ry")

In [ ]:
# Verify pseudopotential file content
print("Pseudopotential file header:")
print("=" * 50)
with open(PSEUDO_DIR / 'Si.upf', 'r') as f:
    for i, line in enumerate(f):
        if i < 20:  # Print first 20 lines
            print(line.rstrip())
        else:
            break

---

## 5. Understanding the QE Input File Structure

A QE `pw.x` input file consists of several **namelists** and **cards**:

### Namelists (Fortran syntax with `&name ... /`)

| Namelist | Purpose |
|----------|----------|
| `&CONTROL` | Job type, directories, output control |
| `&SYSTEM` | Physical system parameters (atoms, electrons, cutoffs) |
| `&ELECTRONS` | SCF convergence parameters |
| `&IONS` | Ionic relaxation (for relax/vc-relax) |
| `&CELL` | Cell optimization (for vc-relax) |

### Cards (Free format)

| Card | Purpose |
|------|----------|
| `ATOMIC_SPECIES` | Element, mass, pseudopotential file |
| `ATOMIC_POSITIONS` | Atomic coordinates |
| `K_POINTS` | k-point mesh or explicit list |
| `CELL_PARAMETERS` | Unit cell vectors (if not using ibrav) |

In [ ]:
# Let's create a template SCF input file for bulk Silicon
# and understand each parameter

scf_input_template = '''&CONTROL
    calculation = 'scf'          ! Type: 'scf', 'relax', 'vc-relax', 'bands', 'nscf'
    prefix = 'silicon'           ! Prefix for output files
    outdir = './tmp'             ! Directory for temporary files
    pseudo_dir = '{pseudo_dir}'  ! Directory containing pseudopotentials
    verbosity = 'high'           ! Output detail: 'low' or 'high'
    tprnfor = .true.             ! Print forces
    tstress = .true.             ! Print stress tensor
/

&SYSTEM
    ibrav = 2                    ! Bravais lattice: 2 = FCC
    celldm(1) = 10.26            ! Lattice parameter in Bohr (a = 5.43 Å)
    nat = 2                      ! Number of atoms in unit cell
    ntyp = 1                     ! Number of atom types
    ecutwfc = 30.0               ! Kinetic energy cutoff for wavefunctions (Ry)
    ecutrho = 240.0              ! Kinetic energy cutoff for charge density (Ry)
    occupations = 'smearing'     ! Occupation method for metals/semiconductors
    smearing = 'cold'            ! Smearing type: 'gaussian', 'mp', 'cold', 'mv'
    degauss = 0.01               ! Smearing width (Ry)
/

&ELECTRONS
    conv_thr = 1.0e-8            ! SCF convergence threshold (Ry)
    mixing_beta = 0.7            ! Mixing factor for SCF
    mixing_mode = 'plain'        ! Mixing algorithm
    diagonalization = 'david'    ! Eigenvalue solver
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf          ! Element, atomic mass, pseudopotential file

ATOMIC_POSITIONS {{crystal}}       ! Positions in crystal coordinates
    Si  0.00  0.00  0.00         ! Atom 1 at origin
    Si  0.25  0.25  0.25         ! Atom 2 at (1/4, 1/4, 1/4)

K_POINTS {{automatic}}             ! Automatic Monkhorst-Pack grid
    4 4 4 0 0 0                  ! nx ny nz  sx sy sz (shifts)
'''

print("Example SCF Input File for Bulk Silicon:")
print("=" * 60)
print(scf_input_template.format(pseudo_dir=str(PSEUDO_DIR)))

---

## 6. Crystal Structure: Diamond Cubic Silicon

Silicon has the **diamond cubic** structure:
- Space group: Fd-3m (227)
- Lattice constant: a = 5.43 Å (experimental)
- Basis: 2 atoms at (0,0,0) and (1/4, 1/4, 1/4)

In QE, we can use `ibrav = 2` (FCC) with 2 atoms, which is equivalent to the diamond structure.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Silicon diamond structure
a = 5.43  # Lattice constant in Angstrom

# FCC lattice vectors
lattice_vectors = a * np.array([
    [-0.5, 0.0, 0.5],
    [0.0, 0.5, 0.5],
    [-0.5, 0.5, 0.0]
])

# Basis atoms in crystal coordinates
basis_crystal = np.array([
    [0.00, 0.00, 0.00],
    [0.25, 0.25, 0.25]
])

# Convert to Cartesian coordinates
basis_cartesian = basis_crystal @ lattice_vectors

print("Silicon Diamond Structure")
print("=" * 40)
print(f"\nLattice constant: a = {a} Å")
print(f"                  a = {a / 0.529177:.4f} Bohr")
print(f"\ncelldm(1) = {a / 0.529177:.4f} Bohr")
print("\nLattice vectors (Å):")
for i, vec in enumerate(lattice_vectors):
    print(f"  a{i+1} = [{vec[0]:8.4f}, {vec[1]:8.4f}, {vec[2]:8.4f}]")
print("\nBasis atoms (Cartesian, Å):")
for i, pos in enumerate(basis_cartesian):
    print(f"  Si{i+1} = [{pos[0]:8.4f}, {pos[1]:8.4f}, {pos[2]:8.4f}]")

In [ ]:
# Visualize the unit cell
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Generate atoms in a 2x2x2 supercell for visualization
atoms = []
for i in range(-1, 2):
    for j in range(-1, 2):
        for k in range(-1, 2):
            shift = i * lattice_vectors[0] + j * lattice_vectors[1] + k * lattice_vectors[2]
            for basis in basis_cartesian:
                atoms.append(basis + shift)

atoms = np.array(atoms)

# Plot atoms
ax.scatter(atoms[:, 0], atoms[:, 1], atoms[:, 2], 
           c='steelblue', s=200, alpha=0.8, edgecolors='navy', linewidths=1)

# Draw unit cell edges
origin = np.array([0, 0, 0])
for i in range(3):
    ax.plot3D(*zip(origin, lattice_vectors[i]), 'k-', linewidth=2)
    for j in range(3):
        if i != j:
            ax.plot3D(*zip(lattice_vectors[i], lattice_vectors[i] + lattice_vectors[j]), 'k-', linewidth=1)

ax.set_xlabel('x (Å)', fontsize=12)
ax.set_ylabel('y (Å)', fontsize=12)
ax.set_zlabel('z (Å)', fontsize=12)
ax.set_title('Silicon Diamond Structure (FCC primitive cell)', fontsize=14)
ax.set_box_aspect([1, 1, 1])

plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'silicon_structure.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f"\nFigure saved to: {OUTPUT_DIR / 'silicon_structure.png'}")

---

## 7. Quick Test: Verify QE Works

Let's run a very quick SCF calculation to verify everything is working.

In [ ]:
# Create a test calculation directory
test_dir = OUTPUT_DIR / 'test_scf'
test_dir.mkdir(exist_ok=True)
(test_dir / 'tmp').mkdir(exist_ok=True)

# Write a minimal test input
test_input = f"""&CONTROL
    calculation = 'scf'
    prefix = 'si_test'
    outdir = './tmp'
    pseudo_dir = '{PSEUDO_DIR}'
/
&SYSTEM
    ibrav = 2
    celldm(1) = 10.26
    nat = 2
    ntyp = 1
    ecutwfc = 20.0
    ecutrho = 160.0
/
&ELECTRONS
    conv_thr = 1.0e-6
/
ATOMIC_SPECIES
    Si 28.0855 Si.upf
ATOMIC_POSITIONS {{crystal}}
    Si 0.00 0.00 0.00
    Si 0.25 0.25 0.25
K_POINTS {{automatic}}
    2 2 2 0 0 0
"""

input_file = test_dir / 'si_test.in'
with open(input_file, 'w') as f:
    f.write(test_input)

print(f"Test input written to: {input_file}")
print("\nRunning quick test calculation...")

In [ ]:
# Run the test calculation
import time

start_time = time.time()
result = subprocess.run(
    ['pw.x', '-in', 'si_test.in'],
    capture_output=True,
    text=True,
    cwd=test_dir,
    timeout=120
)
elapsed = time.time() - start_time

# Save output
output_file = test_dir / 'si_test.out'
with open(output_file, 'w') as f:
    f.write(result.stdout)

print(f"Calculation completed in {elapsed:.2f} seconds")
print(f"Output saved to: {output_file}")

# Check for convergence
if 'convergence has been achieved' in result.stdout:
    print("\n✓ SCF CONVERGED SUCCESSFULLY!")
else:
    print("\n✗ SCF did not converge or error occurred")
    if result.stderr:
        print(f"Errors: {result.stderr}")

In [ ]:
# Parse and display key results
def parse_scf_output(output_text):
    """Parse key quantities from pw.x SCF output."""
    results = {}
    
    for line in output_text.split('\n'):
        if '!' in line and 'total energy' in line:
            # Extract total energy
            parts = line.split('=')
            if len(parts) >= 2:
                energy_str = parts[1].strip().split()[0]
                results['total_energy_ry'] = float(energy_str)
                results['total_energy_ev'] = float(energy_str) * 13.605693  # Ry to eV
        
        if 'number of iterations' in line.lower() or 'convergence has been achieved in' in line:
            parts = line.split()
            for i, p in enumerate(parts):
                if p.isdigit():
                    results['n_iterations'] = int(p)
                    break
        
        if 'total magnetization' in line:
            parts = line.split('=')
            if len(parts) >= 2:
                results['magnetization'] = float(parts[1].split()[0])
        
        if 'kinetic-energy cutoff' in line:
            parts = line.split('=')
            if len(parts) >= 2:
                results['ecutwfc'] = float(parts[1].split()[0])
        
        if 'number of k points' in line:
            parts = line.split('=')
            if len(parts) >= 2:
                results['nkpts'] = int(parts[1].split()[0])
    
    return results

results = parse_scf_output(result.stdout)

print("\nSCF Calculation Results:")
print("=" * 40)
if 'total_energy_ry' in results:
    print(f"Total Energy: {results['total_energy_ry']:.8f} Ry")
    print(f"             {results['total_energy_ev']:.6f} eV")
if 'n_iterations' in results:
    print(f"SCF iterations: {results['n_iterations']}")
if 'nkpts' in results:
    print(f"Number of k-points: {results['nkpts']}")
if 'ecutwfc' in results:
    print(f"Wavefunction cutoff: {results['ecutwfc']} Ry")

print("\n✓ Environment verification complete!")
print("You are ready to proceed with the workshop.")

---

## Summary

In this notebook, we have:

1. ✓ Verified Quantum ESPRESSO installation
2. ✓ Set up the directory structure
3. ✓ Downloaded and verified the Silicon pseudopotential
4. ✓ Understood the QE input file structure
5. ✓ Visualized the Silicon crystal structure
6. ✓ Ran a test SCF calculation

### Key Parameters to Remember

| Parameter | Meaning | Typical Values |
|-----------|---------|----------------|
| `ecutwfc` | Wavefunction cutoff | 30-80 Ry (depends on PP) |
| `ecutrho` | Charge density cutoff | 4-12× ecutwfc |
| `K_POINTS` | Brillouin zone sampling | 4×4×4 to 12×12×12 |
| `conv_thr` | SCF convergence | 1e-6 to 1e-10 Ry |

### Next Notebook
→ **02_SCF_Calculation_Basics.ipynb**: Deep dive into SCF calculations and understanding the output